In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 25kB/s 
     |████████████████████████████████| 5.9MB 19.3MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [23]:
import sys
sys.path.append('/content/drive/My Drive')
import torch
from tqdm.notebook import tqdm
from torch import nn
from torch import optim
from torchvision import models
from utils.dataloader_image_classification_colab import ImageTransform, make_datapath_list, HymenopteraDataset

# アリとハチの画像へのファイルパスのリストを作成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

# Datasetを作成
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(train_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = HymenopteraDataset(val_list, transform=ImageTransform(size, mean, std), phase='val')

# DataLoaderを作成
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


/content/drive/My Drive/data/hymenoptera_data/train/**/*.jpg
/content/drive/My Drive/data/hymenoptera_data/val/**/*.jpg


In [21]:
print(val_dataset.__getitem__(0)[0].size())
print(val_dataset.__getitem__(0)[1])

  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([3, 224, 224])
1


KeyboardInterrupt: ignored

In [3]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの２つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [5]:
# ファインチューニングで学習させるパラメータを、変数params_to_updateの１～３に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight']
update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print('params_to_update_1に格納 : ', name)
  
  elif name in update_param_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    print('params_to_update_2に格納 : ', name)

  elif name in update_param_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    print('params_to_update_3に格納 : ', name)
  
  else:
    param.requires_grad = False
    print('勾配計算なし : ', name)

params_to_update_1に格納 :  features.0.weight
params_to_update_1に格納 :  features.0.bias
params_to_update_1に格納 :  features.2.weight
params_to_update_1に格納 :  features.2.bias
params_to_update_1に格納 :  features.5.weight
params_to_update_1に格納 :  features.5.bias
params_to_update_1に格納 :  features.7.weight
params_to_update_1に格納 :  features.7.bias
params_to_update_1に格納 :  features.10.weight
params_to_update_1に格納 :  features.10.bias
params_to_update_1に格納 :  features.12.weight
params_to_update_1に格納 :  features.12.bias
params_to_update_1に格納 :  features.14.weight
params_to_update_1に格納 :  features.14.bias
params_to_update_1に格納 :  features.17.weight
params_to_update_1に格納 :  features.17.bias
params_to_update_1に格納 :  features.19.weight
params_to_update_1に格納 :  features.19.bias
params_to_update_1に格納 :  features.21.weight
params_to_update_1に格納 :  features.21.bias
params_to_update_1に格納 :  features.24.weight
params_to_update_1に格納 :  features.24.bias
params_to_update_1に格納 :  features.26.weight
params_to_update_1

In [6]:
# 最適化手法の設定
optimizer = optim.SGD([
  {'params': params_to_update_1, 'lr': 1e-4},
  {'params': params_to_update_2, 'lr': 5e-4},
  {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [7]:
# モデルを学習させるための関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
  # 初期設定
  # GPUが使えるかを確認
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  print('使用デバイス : ', device)

  # ネットワークをGPUへ
  net.to(device)

  # ネットワークがある程度固定であれば、高速化させる
  torch.backends.cudnn.benchmark = True

  # epochのループ
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')

    # epochごとの訓練と検証のループ
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train()
      else:
        net.eval()

      epoch_loss = 0.0
      epoch_corrects = 0

      # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
      if (epoch == 0) and (phase == 'train'):
        continue
      
      # データローダーからミニバッチを取り出すループ
      for inputs, labels in tqdm(dataloaders_dict[phase]):

        # GPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizerの初期化
        optimizer.zero_grad()

        # 順伝播(forward)計算
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs)
          loss = criterion(outputs, labels) # 損失を計算
          _, preds = torch.max(outputs, 1)  # ラベルを予測

          # 訓練時はバックプロパゲーション
          if phase == 'train':
            loss.backward()
            optimizer.step()

          # 結果の計算
          epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
          # 正解数の合計を更新
          epoch_corrects += torch.sum(preds == labels.data)

      # epochごとのlossと正解率を表示
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


In [24]:
# 学習・検証を実行
num_epochs = 2
train_model(net, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs)

使用デバイス :  cuda:0
Epoch 1/2
-------------



val Loss: 0.7900 Acc: 0.4118
Epoch 2/2
-------------



train Loss: 0.4900 Acc: 0.7325



val Loss: 0.1554 Acc: 0.9542
